In [21]:
import os
import glob

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

import ray

# 바이살라 루프트 자료 병합

In [22]:
LuftFileList = glob.glob("../DATA/MWIS_2023_NDJF_OBS/*MARWIS*csv")
VaisalaFileList = glob.glob("../DATA/MWIS_2023_NDJF_OBS/*move4*csv")

luft_total_list = []
for luft in LuftFileList:
    luft_df = pd.read_csv(luft, low_memory=False)
    luft_total_list.append(luft_df)
luft_total_df = pd.concat(luft_total_list)

vi_total_list = []
for vi in VaisalaFileList:
    vi_df = pd.read_csv(vi, low_memory=False)
    vi_total_list.append(vi_df)
vi_total_df = pd.concat(vi_total_list)

def vaisala_road(s_status):
    vaisala_road_state = {
        '1': "Dry",
        '2': "Moist",
        '3': "Wet",
        '5': "Frost",
        '6': "Snow",
        '7': "Ice",
        '9': "Slush",
    }

    if s_status not in vaisala_road_state:
        return "error"
    return vaisala_road_state[s_status]

vi_total_df["s_status_txt"] = vi_total_df["s_status"].apply(vaisala_road)

drop_column_list = ['gdirection1','gdirection2','digitalcomp_x','digitalcomp_y','digitalcomp_z',
                    'ta2','rh2','loggerta', 'batteryvolt', 'rev1', 'rev2']
vi_total_df.drop(drop_column_list, axis = 1, inplace=True)

# Extract components of the date and time
year = vi_total_df['gdate'].astype(str).str[-6:-4].apply(lambda x: '20' + x if len(x) == 2 else x)  # Handling two-digit year
month = vi_total_df['gdate'].astype(str).str[-4:-2]
day = vi_total_df['gdate'].astype(str).str[-2:]

hour = vi_total_df['gtime'].astype(str).str[-6:-4].str.zfill(2)
minute = vi_total_df['gtime'].astype(str).str[-4:-2]
second = vi_total_df['gtime'].astype(str).str[-2:]

# Combine extracted components into a datetime string and convert to datetime
vi_total_df['TIMESTAMP'] = pd.to_datetime(
    year + '-' + month + '-' + day + ' ' + hour + ':' + minute + ':' + second,
    format='%Y-%m-%d %H:%M:%S'
)

vi_total_df['TIMESTAMP'] = pd.to_datetime(vi_total_df['TIMESTAMP'])
luft_total_df['TIMESTAMP'] = pd.to_datetime(luft_total_df['TIMESTAMP'])

total_df = pd.merge(vi_total_df, luft_total_df, on='TIMESTAMP', how='outer', suffixes=('_vi', '_luft'))

total_df.replace({
    '/': '',
    '//////': np.nan,
    '////': np.nan,
    '///////': np.nan
}, inplace=True)


total_df.to_csv("../DATA/CONCAT/luft_vi_2023_2024.csv", index=None)



# 센서 자료 AWS자료 병합

In [28]:
from scipy.spatial import KDTree

total_df['glatitude']=total_df['glatitude'].ffill() 
total_df['glongitude']=total_df['glongitude'].ffill() 

# AWS 정보 데이터프레임 읽기 및 k-d 트리 생성
aws_info_df = pd.read_csv("../DATA/AWS/META_관측지점정보.csv", encoding='cp949')
aws_coord = aws_info_df.drop_duplicates(subset=["지점"])[['지점', '위도', '경도']]
aws_coord.columns = ['site', 'latitude', 'longitude']

# 위도와 경도 점들을 배열로 생성
points = np.array(list(zip(aws_coord['latitude'], aws_coord['longitude'])))
tree = KDTree(points)

# 질의 점들 생성
query_points = total_df[['glatitude', 'glongitude']].to_numpy()

# 가장 가까운 점들 찾기
distances, indices = tree.query(query_points)

# 가장 가까운 점의 사이트 정보 가져오기
nearest_sites = aws_coord.iloc[indices]['site'].values

# total_df에 AWS 사이트 정보 추가
total_df['aws_site'] = nearest_sites

In [16]:
asw_stn_list = total_df['aws_site'].unique()

In [6]:
total_df

,vehicle,gdate,gtime,glatitude,glongitude,galtitude,vsp,rws,rwd,tws,...,R_Dew,R_Hum,R_WH,R_Sta,R_Ice,R_Fric,R_Sta_txt,BattV,PTemp,aws_site
0,A,231117.0,65401.0,37.516632,127.439939,55.2,0.0,0.5,336.00,0.49,...,-1.210844,92.73611,0.041476,5.0,0.000000e+00,0.808356,Chemically wet,11.99,17.76652,326
1,A,231117.0,65402.0,37.516632,127.439939,55.2,0.0,0.5,337.00,0.46,...,-1.210844,93.70230,0.039128,5.0,0.000000e+00,0.808801,Chemically wet,12.00,17.76652,326
2,A,231117.0,65403.0,37.516632,127.439939,55.2,0.0,0.4,334.00,0.39,...,-1.219853,93.95007,0.041893,5.0,0.000000e+00,0.808278,Chemically wet,12.00,17.76652,326
3,NaN,NaN,NaN,37.516632,127.439939,NaN,NaN,NaN,NaN,NaN,...,-1.219853,93.49349,0.041732,5.0,0.000000e+00,0.808308,Chemically wet,12.00,17.76652,326
4,A,231117.0,65405.0,37.516633,127.439938,55.4,0.0,0.3,328.00,0.33,...,-1.210038,94.23958,0.040386,5.0,0.000000e+00,0.808561,Chemically wet,11.99,17.79138,326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
943543,A,240223.0,124001.0,37.460904,126.890301,21.0,11.2,11.4,357.00,4.17,...,-2.062749,30.24125,0.026036,1.0,3.082857e-44,0.811540,Damp,11.93,26.96393,417
943544,A,240223.0,124002.0,37.461070,126.890166,20.2,10.9,11.5,357.00,4.17,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,417
943545,A,240223.0,124003.0,37.461148,126.890090,20.6,10.4,11.5,357.00,4.00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,417
943546,A,240223.0,124004.0,37.461231,126.890008,23.1,11.1,11.4,356.00,4.13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,417


In [ ]:
# luft_total_df.to_csv("../DATA/CONCAT/luft_2023_2024.csv", index=None)
# vi_total_df.to_csv("../DATA/CONCAT/visala_2023_2024.csv", index=None)

In [19]:
import ray

asw_stn_list = total_df['aws_site'].unique()

aws_file_dir = "C:/Users/user/Desktop/RoadAnalysis/DATA/AWS/202311~202403/"
aws_file_list = glob.glob(os.path.join(aws_file_dir,"**/AWS*"), recursive=True)
# Ray 초기화
ray.init()

@ray.remote
def process_file(file,asw_stn_list):
    df = pd.read_csv(file, sep="#", header=None,encoding='cp949',engine='python')
    df.columns = ['STN_ID', 'TM', 'LAT', 'LON', 'HT', 'WD', 'WS', 'TA', 'HM', 'PA', 'PS', 'RN_YN',
                  'RN_1HR', 'RN_DAY', 'RN_15M', 'RN_60M', 'WD_INS', 'WS_INS', "END"]
    df = df[df['STN_ID'].isin(asw_stn_list)]
    return df

# 병렬로 파일 처리
futures = [process_file.remote(file,asw_stn_list) for file in aws_file_list]
dfs_list = ray.get(futures)

# 데이터프레임 병합
dfs = pd.concat(dfs_list)

# Ray 종료
ray.shutdown()

# 결과 출력 또는 저장
# print(dfs)
dfs.to_csv("merged_output.csv", index=False)


2024-07-24 09:59:26,506	INFO worker.py:1779 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


# AWS 자료와 센서 자료 결합

In [72]:
aws_df = pd.read_csv("../DATA/AWS/merged_output.csv")

In [73]:
aws_df

,STN_ID,TM,LAT,LON,HT,WD,WS,TA,HM,PA,PS,RN_YN,RN_1HR,RN_DAY,RN_15M,RN_60M,WD_INS,WS_INS,END
0,326,202311010000,37.5455,127.6113,191.01,3044,4,95,997,9982,10213,0,0,0,0,0,3217,5,=
1,377,202311010000,37.3952,126.8919,28.00,604,4,170,849,10164,10197,0,0,0,0,0,982,8,=
2,417,202311010000,37.4655,126.9002,45.00,1599,19,172,817,10145,10198,0,0,0,0,0,1642,28,=
3,515,202311010000,37.0845,126.7739,10.47,1715,25,167,897,-997,-997,0,0,0,0,0,1941,35,=
4,533,202311010000,37.3278,127.4807,82.00,0,0,115,912,-997,-997,0,0,0,0,0,1772,0,=
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9411835,899,202403312359,35.2235,128.5822,33.30,2675,10,101,852,10069,10109,0,0,0,0,0,2792,14,=
9411836,903,202403312359,35.4039,128.5149,14.86,489,7,63,984,10088,10106,0,0,0,0,0,558,8,=
9411837,919,202403312359,35.5514,128.4767,30.00,0,0,67,825,10080,10117,0,0,0,0,0,2223,0,=
9411838,920,202403312359,35.2961,128.3994,8.39,1050,16,62,979,10100,10110,0,0,0,0,0,1053,19,=


In [69]:
# Rename the column 'STN_ID' to 'aws_site' in aws_df
aws_df.rename(columns={"STN_ID": "aws_site"}, inplace=True)

# Convert 'aws_site' to integer type in aws_df
aws_df["aws_site"] = aws_df["aws_site"].astype(int)

# Convert 'TM' to datetime and create 'min_time' column in aws_df
aws_df['min_time'] = pd.to_datetime(aws_df['TM'], format='%Y%m%d%H%M')
aws_df['min_time'] = aws_df['min_time'].dt.strftime('%Y-%m-%d %H:%M')

# Convert 'TIMESTAMP' to datetime and create 'min_time' column in total_df
total_df['min_time'] = pd.to_datetime(total_df['TIMESTAMP'], format='%Y%m%d%H%M')
total_df['min_time'] = total_df['min_time'].dt.strftime('%Y-%m-%d %H:%M')

# Convert 'aws_site' to integer type in total_df
total_df["aws_site"] = total_df["aws_site"].astype(int)

# Print results for verification
print(aws_df)
print(total_df)


         aws_site            TM      LAT       LON      HT    WD  WS   TA  \
0             326  202311010000  37.5455  127.6113  191.01  3044   4   95   
1             377  202311010000  37.3952  126.8919   28.00   604   4  170   
2             417  202311010000  37.4655  126.9002   45.00  1599  19  172   
3             515  202311010000  37.0845  126.7739   10.47  1715  25  167   
4             533  202311010000  37.3278  127.4807   82.00     0   0  115   
...           ...           ...      ...       ...     ...   ...  ..  ...   
9411835       899  202403312359  35.2235  128.5822   33.30  2675  10  101   
9411836       903  202403312359  35.4039  128.5149   14.86   489   7   63   
9411837       919  202403312359  35.5514  128.4767   30.00     0   0   67   
9411838       920  202403312359  35.2961  128.3994    8.39  1050  16   62   
9411839       991  202403312359  35.8028  128.4456   27.50  3054  16   76   

          HM     PA  ...  RN_YN  RN_1HR  RN_DAY  RN_15M  RN_60M  WD_INS  \


In [64]:
aws_df['TIMESTAMP']

0         2023-11-01 00:00:00
1         2023-11-01 00:00:00
2         2023-11-01 00:00:00
3         2023-11-01 00:00:00
4         2023-11-01 00:00:00
                  ...        
9411835   2024-03-31 23:59:00
9411836   2024-03-31 23:59:00
9411837   2024-03-31 23:59:00
9411838   2024-03-31 23:59:00
9411839   2024-03-31 23:59:00
Name: TIMESTAMP, Length: 9411840, dtype: datetime64[ns]

In [63]:
total_df['TIMESTAMP']

0        2023-11-17 06:54:01
1        2023-11-17 06:54:02
2        2023-11-17 06:54:03
3        2023-11-17 06:54:04
4        2023-11-17 06:54:05
                 ...        
943543   2024-02-23 12:40:01
943544   2024-02-23 12:40:02
943545   2024-02-23 12:40:03
943546   2024-02-23 12:40:04
943547   2024-02-23 12:40:05
Name: TIMESTAMP, Length: 943548, dtype: datetime64[ns]

In [70]:
df_last = pd.merge(left=total_df,right=aws_df, on = ["aws_site",'min_time'], how = 'left')

In [71]:
df_last.to_csv("test.csv",index=None)